In [94]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time
import dateparser
import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd


In [95]:
FIRST_YEAR = 'https://www.myhora.com/%E0%B8%9B%E0%B8%8F%E0%B8%B4%E0%B8%97%E0%B8%B4%E0%B8%99/%E0%B8%A7%E0%B8%B1%E0%B8%99%E0%B8%AB%E0%B8%A2%E0%B8%B8%E0%B8%94-%E0%B8%9E.%E0%B8%A8.2564.aspx'

In [96]:
def getPageDate(link):
    web = requests.get(link)
    soup = BeautifulSoup(web.text, 'html.parser')
    all = soup.find_all('div', class_='dxl')
    return all



In [97]:
def getAllPage(first_page):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(first_page)
    iterations = 1
    lastpage = '0000'
    all = []
    while True:
        try:
            iterations += 1
            this_page = driver.find_element(By.XPATH, '//h1[@class="main-title"]').text
            if this_page[-4:] < lastpage[-4:]:
                break
            print(this_page)
            all += getPageDate(driver.current_url)
            next_page = driver.find_element(By.XPATH, '//div[@class="btn-yellow ml-10"]').click()
            WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element((By.XPATH, '//h1[@class="main-title"]'),str(iterations)))
            lastpage = this_page
        except:
            break
    driver.close()
    return all

In [98]:
def collect(first_page):
    all = getAllPage(first_page)
    date = []
    holiday_title  = []
    affair_holiday = []
    bank_holiday = []


    for i in all:
        date.append(i.find('div', class_='dxl-dmy').text)
        holiday_title.append(i.find('div', class_='dxl-dn').text)
        affair = i.find('div', class_='dxl-dn2')
        if affair.find('font') is None:
            affair_holiday.append(1)
        else:
            affair_holiday.append(0)
        bank = i.find('div', class_='dxl-dn3')
        if bank.find('font') is None:
            bank_holiday.append(1)
        else:
            bank_holiday.append(0)
    df = pd.DataFrame({'date':date, 'holiday_title':holiday_title, 'affair_holiday':affair_holiday, 'bank_holiday':bank_holiday})
    return df


In [112]:
df = collect(FIRST_YEAR)

วันหยุด 2564/2021
วันหยุด 2565/2022
วันหยุด 2566/2023
วันหยุด 2567/2024


In [113]:
df

,date,holiday_title,affair_holiday,bank_holiday
0,1มกราคม2564,วันขึ้นปีใหม่,1,1
1,12กุมภาพันธ์2564,วันตรุษจีน,1,1
2,26กุมภาพันธ์2564,วันมาฆบูชา,1,1
3,6เมษายน2564,วันจักรี,1,1
4,12เมษายน2564,วันสงกรานต์ (ครม.),1,0
...,...,...,...,...
102,14ตุลาคม2567,วันหยุดชดเชย,1,1
103,23ตุลาคม2567,วันปิยมหาราช,1,1
104,5ธันวาคม2567,"วันคล้ายวันเฉลิมฯ ร.9 , วันพ่อ",1,1
105,10ธันวาคม2567,วันรัฐธรรมนูญ,1,1


In [114]:
df['date'] = df['date'].apply(lambda x: dateparser.parse(x).replace(year=(dateparser.parse(x).year)-543))
df['weekday'] = df['date'].apply(lambda x: x.weekday())

In [116]:
df.to_csv('holiday.csv', index=False)